# Module 2. Personalize 솔류션 생성 및 평가 지표 확인하기

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.

* 레서피(알고리즘) 선택 및 솔류션 생성
* 솔류션 버전 생성
* 솔류션 평가 지표 얻기



## 라이브러리 임포트

파이썬에는 광범위한 라이브러리 모음이 포함되어 있으며, 본 핸즈온을 위해서 핵심 데이터 과학 도구인 boto3 (AWS SDK) 및 Pandas/Numpy와 같은 라이브러리를 가져와야 합니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
suffix = str(np.random.uniform())[4:9]

## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

이 LAB에서 아래와 같은 솔루션을 생성하여 성능을 비교하여 봅니다.

    1) HRNN
    2) HRNN Coldstart
    3) SIMS
    4) Personalized Ranking

이 프로세스의 완료는 실제로 40분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

###  레시피 리스트 확인

In [5]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims


###  Popularity Count 솔루션 생성 

In [6]:
# Recipe 선택 
popularity_recipe_arn = "arn:aws:personalize:::recipe/aws-popularity-count"

In [7]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-popularity-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = popularity_recipe_arn,
)

popularity_recipe_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-popularity-30524",
  "ResponseMetadata": {
    "RequestId": "34350345-0e8e-4d6c-89a6-0c51c73e9226",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:37 GMT",
      "x-amzn-requestid": "34350345-0e8e-4d6c-89a6-0c51c73e9226",
      "content-length": "96",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [8]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = popularity_recipe_arn
)

popularity_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-popularity-30524/40427de0",
  "ResponseMetadata": {
    "RequestId": "dc4ead05-c3f2-43f0-b6b5-98de74edd0e3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:37 GMT",
      "x-amzn-requestid": "dc4ead05-c3f2-43f0-b6b5-98de74edd0e3",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN 솔루션 생성

In [9]:
 # Recipe 선택 
hrnn_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

In [10]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-hrnn-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_recipe_arn,
)

hrnn_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-30524",
  "ResponseMetadata": {
    "RequestId": "2a60fa7b-0384-4144-917f-73c93f9dcacb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:38 GMT",
      "x-amzn-requestid": "2a60fa7b-0384-4144-917f-73c93f9dcacb",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [11]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

hrnn_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-30524/b849addd",
  "ResponseMetadata": {
    "RequestId": "d1aa52da-c4bd-4351-8d90-eec4a9e71a5e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:40 GMT",
      "x-amzn-requestid": "d1aa52da-c4bd-4351-8d90-eec4a9e71a5e",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN ColdStart 솔루션 생성

In [12]:
 # Recipe 선택 
hrnn_coldstart_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-coldstart"

In [13]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-hrnn-coldstart-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = hrnn_coldstart_recipe_arn,
    solutionConfig = {
        "featureTransformationParameters" : {
            'cold_start_max_duration' : '5',
            'cold_start_relative_from' : 'latestItem',
            'cold_start_max_interactions':'15'
        }
    }
)

hrnn_coldstart_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-coldstart-30524",
  "ResponseMetadata": {
    "RequestId": "1466bccd-9879-4c76-b9c3-b5f0ea141811",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:42 GMT",
      "x-amzn-requestid": "1466bccd-9879-4c76-b9c3-b5f0ea141811",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [14]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_coldstart_solution_arn
)

hrnn_coldstart_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-coldstart-30524/f5f0d400",
  "ResponseMetadata": {
    "RequestId": "9b8d47ba-1dcc-4b2a-9b56-c05ae71d19ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:43 GMT",
      "x-amzn-requestid": "9b8d47ba-1dcc-4b2a-9b56-c05ae71d19ca",
      "content-length": "116",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS 솔루션 생성 

In [15]:
 # Recipe 선택 
sims_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

In [16]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-sims-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = sims_recipe_arn
)

sims_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-sims-30524",
  "ResponseMetadata": {
    "RequestId": "51df39dc-2878-4043-932d-3519a021565e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:48 GMT",
      "x-amzn-requestid": "51df39dc-2878-4043-932d-3519a021565e",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [17]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)
sims_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-sims-30524/75728970",
  "ResponseMetadata": {
    "RequestId": "926b4dc2-bc4c-45e8-af69-14bb623dfbd8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:49 GMT",
      "x-amzn-requestid": "926b4dc2-bc4c-45e8-af69-14bb623dfbd8",
      "content-length": "106",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalize Ranking 솔루션 생성 

In [18]:
 # Recipe 선택 
ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

In [19]:
# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Demo-ranking-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = ranking_recipe_arn
)

ranking_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-ranking-30524",
  "ResponseMetadata": {
    "RequestId": "fab2c60e-4585-4b22-b13e-a1642767a1de",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:53 GMT",
      "x-amzn-requestid": "fab2c60e-4585-4b22-b13e-a1642767a1de",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [20]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = ranking_solution_arn
)
ranking_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-ranking-30524/ece21761",
  "ResponseMetadata": {
    "RequestId": "46131179-0d3e-479e-8b84-1bf8ef65bab6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 00:11:58 GMT",
      "x-amzn-requestid": "46131179-0d3e-479e-8b84-1bf8ef65bab6",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 모델 생성 확인 하기 
아래 코드 실행하여 모델 생성 여부를 확인합니다. 

In [23]:
%%time


max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    ##Popularity Status
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = popularity_solution_version_arn
    )  
    status_pop = describe_solution_version_response["solutionVersion"]["status"]
    print("Popularity SolutionVersion: {}".format(status_pop))
    
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_solution_version_arn
    )  
    status_hrnn = describe_solution_version_response["solutionVersion"]["status"]
    print("HRNN SolutionVersion: {}".format(status_hrnn))
    
    #hrnn_coldstart status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_coldstart_solution_version_arn
    )
    status_hrnn_cs = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN ColdStart SolutionVersion: {}".format(status_hrnn_cs))
    
    #sims status
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = sims_solution_version_arn
    )
    status_sims = describe_solution_version_response["solutionVersion"]["status"]
    print("Sims SolutionVersion: {}".format(status_sims))
    
    #ranking status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = ranking_solution_version_arn
    )
    status_ranking= describe_solution_version_response["solutionVersion"]["status"]    
    print("Ranking SolutionVersion: {}".format(status_ranking))    
    
    if (status_pop == "ACTIVE" or status_pop == "CREATE FAILED") &\
       (status_hrnn == "ACTIVE" or status_hrnn == "CREATE FAILED") &\
       (status_hrnn_cs == "ACTIVE" or status_hrnn_cs == "CREATE FAILED") &\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED") &\
       (status_ranking == "ACTIVE" or status_ranking == "CREATE FAILED"):
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All solution creation completed")

Popularity SolutionVersion: ACTIVE
HRNN SolutionVersion: ACTIVE
HRNN ColdStart SolutionVersion: ACTIVE
Sims SolutionVersion: ACTIVE
Ranking SolutionVersion: ACTIVE
All solution creation completed
CPU times: user 21.9 ms, sys: 0 ns, total: 21.9 ms
Wall time: 209 ms


## 솔루션 평가 지표 얻기

이번 파트에서는 Amazon Personalize에서 기본으로 제공하는 솔루션에 대한 평가 지표를 확인해 봅니다. 
Amazon Personalize에서는 평가 지표를 생성하기 위해 약 랜덤으로 10% 사용자의 interaction data를 테스트 용으로 활용합니다. 

아래 이미지는 Amazon Personalize가 데이터를 분리하는 방법을 보여줍니다. 사용자가 10 명이고 각각 10 개의 상호 작용이있는 경우 (여기에서 원은 Interaction data를 나타냄) 타임 스탬프를 기준으로 가장 오래된 것부터 최신 것까지 나열된 것입니다. Amazon Personalize는 사용자의 90 % (파란색 원)의 모든 Interaction 데이터를 사용하여 솔루션 버전을 훈련시키고 나머지 10 %는 평가를 위해 사용합니다. 나머지 10 %의 각 사용자에 대해 Interaction data (녹색 원)의 90 %가 훈련 된 모델의 입력값으로 사용됩니다. 데이터의 나머지 10 % (주황색 원)는 모델에서 생성 된 추천 결과물과 비교되고 평가 지표를 계산하는 데 사용됩니다.



![personalize metrics](static/imgs/personalize_metrics.png)

[솔류션 평가 지표 정의](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)
는 개발자 문서의 링크 참조 바랍니다. 또한 이 링크 [솔류션 평가 정의 예제](http://francescopochetti.com/recommend-expedia-hotels-with-amazon-personalize-the-magic-of-hierarchical-rnns/) 의 페이지 맨 아래 쪽을 보시면 조금 더 직관적인 그림을 보실 수 있습니다.
 <br>
또한 reciprocal_rank_at_5, normalized_discounted_cumulative_gain_at_5,precision_at_5 의 예제는 아래와 같습니다. 
* Exmaple
    * 5 개의 추천리스트를 제공했고, 이 중에 2번째와 5번째가 실제 데이타와 일치 했다고 하면, 쉽게 이렇게 [0,1,0,0,1] 표시 할 수 있습니다.
        * reciprocal_rank
            * 1/2 (0.5) # 가장 빠른 순서의 하나만을 선택 합니다
        * normalized_discounted_cumulative_gain_at_5
            * (1/log(1+2) + 1/log(1+5)) / (1/log(1+1) + 1/log(1+2)) = 0.6241
        * precision_at_5
            * 2/5 (0.4)



#### 조금더 상세하고 Custum 평가 지표를 얻기 위해서 이전에 분리해둔 테스트 데이터를 가지고 캠페인 생성 후 별도 테스트를 진행하도록 합니다.



In [24]:
metrics=[]

def build_metric_matrix(solution,response):
    metrics.append([solution,
                response['metrics']['coverage'],
                response['metrics']['mean_reciprocal_rank_at_25'],
                response['metrics']['normalized_discounted_cumulative_gain_at_5'],
                response['metrics']['normalized_discounted_cumulative_gain_at_10'],
                response['metrics']['normalized_discounted_cumulative_gain_at_25'],
                response['metrics']['precision_at_5'],
                response['metrics']['precision_at_10'],
                response['metrics']['precision_at_25']])

In [25]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

build_metric_matrix('hrnn',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-30524/b849addd",
  "metrics": {
    "coverage": 0.6415,
    "mean_reciprocal_rank_at_25": 0.2604,
    "normalized_discounted_cumulative_gain_at_10": 0.2542,
    "normalized_discounted_cumulative_gain_at_25": 0.3458,
    "normalized_discounted_cumulative_gain_at_5": 0.1992,
    "precision_at_10": 0.0917,
    "precision_at_25": 0.0681,
    "precision_at_5": 0.111
  },
  "ResponseMetadata": {
    "RequestId": "b6801d2f-ebcd-4fac-8499-bd618eeca335",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 03:50:31 GMT",
      "x-amzn-requestid": "b6801d2f-ebcd-4fac-8499-bd618eeca335",
      "content-length": "403",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [26]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = hrnn_coldstart_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('hrnn_coldstart',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-hrnn-coldstart-30524/f5f0d400",
  "metrics": {
    "coverage": 0.2479,
    "mean_reciprocal_rank_at_25": 0.0062,
    "normalized_discounted_cumulative_gain_at_10": 0.0087,
    "normalized_discounted_cumulative_gain_at_25": 0.0117,
    "normalized_discounted_cumulative_gain_at_5": 0.0075,
    "precision_at_10": 0.0013,
    "precision_at_25": 0.001,
    "precision_at_5": 0.0019
  },
  "ResponseMetadata": {
    "RequestId": "8815f163-9165-41de-beca-2f4ab66efe55",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 03:50:31 GMT",
      "x-amzn-requestid": "8815f163-9165-41de-beca-2f4ab66efe55",
      "content-length": "413",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = sims_solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('sims',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-sims-30524/75728970",
  "metrics": {
    "coverage": 0.7378,
    "mean_reciprocal_rank_at_25": 0.1713,
    "normalized_discounted_cumulative_gain_at_10": 0.1896,
    "normalized_discounted_cumulative_gain_at_25": 0.2533,
    "normalized_discounted_cumulative_gain_at_5": 0.1522,
    "precision_at_10": 0.0578,
    "precision_at_25": 0.0425,
    "precision_at_5": 0.0724
  },
  "ResponseMetadata": {
    "RequestId": "9efad7f2-3c86-43c0-9253-e4b898cbf941",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 03:50:32 GMT",
      "x-amzn-requestid": "9efad7f2-3c86-43c0-9253-e4b898cbf941",
      "content-length": "404",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [28]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = ranking_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('ranking',get_solution_metrics_response)

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-ranking-30524/ece21761",
  "metrics": {
    "coverage": 0.0136,
    "mean_reciprocal_rank_at_25": 0.0991,
    "normalized_discounted_cumulative_gain_at_10": 0.1277,
    "normalized_discounted_cumulative_gain_at_25": 0.1518,
    "normalized_discounted_cumulative_gain_at_5": 0.1056,
    "precision_at_10": 0.0277,
    "precision_at_25": 0.0168,
    "precision_at_5": 0.0354
  },
  "ResponseMetadata": {
    "RequestId": "a3d1f5c0-3ee0-4fa9-94cb-19e3ecc002db",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 03:50:32 GMT",
      "x-amzn-requestid": "a3d1f5c0-3ee0-4fa9-94cb-19e3ecc002db",
      "content-length": "407",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [29]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = popularity_solution_version_arn 
)

print(json.dumps(get_solution_metrics_response, indent=2))
build_metric_matrix('popularity',get_solution_metrics_response)


{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:870180618679:solution/Demo-popularity-30524/40427de0",
  "metrics": {
    "coverage": 0.0136,
    "mean_reciprocal_rank_at_25": 0.0522,
    "normalized_discounted_cumulative_gain_at_10": 0.0603,
    "normalized_discounted_cumulative_gain_at_25": 0.1021,
    "normalized_discounted_cumulative_gain_at_5": 0.0382,
    "precision_at_10": 0.017,
    "precision_at_25": 0.0156,
    "precision_at_5": 0.0156
  },
  "ResponseMetadata": {
    "RequestId": "6b84ee7a-f075-4650-acc7-103b314cc097",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 14 Jul 2020 03:50:33 GMT",
      "x-amzn-requestid": "6b84ee7a-f075-4650-acc7-103b314cc097",
      "content-length": "409",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [30]:
metrics=pd.DataFrame(metrics,columns=['recipe','coverage','mrr@25','ndcg@5','ndcg@10','ndcg@25','p@5','p@10','p@25'])

metrics

,recipe,coverage,mrr@25,ndcg@5,ndcg@10,ndcg@25,p@5,p@10,p@25
0,hrnn,0.6415,0.2604,0.1992,0.2542,0.3458,0.1110,0.0917,0.0681
1,hrnn_coldstart,0.2479,0.0062,0.0075,0.0087,0.0117,0.0019,0.0013,0.0010
2,sims,0.7378,0.1713,0.1522,0.1896,0.2533,0.0724,0.0578,0.0425
3,ranking,0.0136,0.0991,0.1056,0.1277,0.1518,0.0354,0.0277,0.0168
4,popularity,0.0136,0.0522,0.0382,0.0603,0.1021,0.0156,0.0170,0.0156


## 리뷰

이번 실습에서는 Amazon Personalize에서 제공하는 다양한 솔루션을 생성하고 Amazon Personalize에서 제공하는 솔루션 버전을 생성해 보았습니다.다음 LAB에서는 (`3.Deploying_Campaigns.ipynb`) 방금 생성한 솔루션을 서비스에 배포 하기 위해 Campaing을 배포해 보도록 합니다. 


In [31]:
%store hrnn_solution_version_arn
%store hrnn_solution_arn
%store hrnn_coldstart_solution_version_arn
%store hrnn_coldstart_solution_arn
%store sims_solution_version_arn
%store sims_solution_arn
%store ranking_solution_version_arn
%store ranking_solution_arn


Stored 'hrnn_solution_version_arn' (str)
Stored 'hrnn_solution_arn' (str)
Stored 'hrnn_coldstart_solution_version_arn' (str)
Stored 'hrnn_coldstart_solution_arn' (str)
Stored 'sims_solution_version_arn' (str)
Stored 'sims_solution_arn' (str)
Stored 'ranking_solution_version_arn' (str)
Stored 'ranking_solution_arn' (str)
